In [1]:
import os 
from pprint import pprint

Check that output from other examples has been generated 

In [2]:
from pathlib2 import Path

In [3]:
need_files = ['methane.xyz']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run structures.ipynb")
        os.system("jupyter nbconvert --to python  structures.ipynb")
        os.system("python structures.py")

In [ ]:
need_files = ['ethane.xyz']
for f in need_files:
    path = Path(f)
    if not path.is_file():
        print("Need to run buildingblocks.ipynb")
        os.system("jupyter nbconvert --to python  buildingblocks.ipynb")
        os.system("python buildingblocks.py")

In [ ]:
import streamm

Now let's create project and resource to keep track of our work

In [ ]:
nwchem_example = streamm.Project('nwchem_example')
res_local = streamm.Resource('local')

In [ ]:
res_local.make_dir()

Update relative location of templates directory 

In [ ]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

Make sure this is the location of the templates directory that comes with the streamm git repository  https://github.com/NREL/streamm-tools

In [ ]:
print res_local.dir['templates']

Create the local directories that will store our files 

In [ ]:
nwchem_example.make_dir()

Tell the project about our directories 

In [ ]:
nwchem_example.set_resource(res_local)

Read in the methane.xyz file created in the structure.ipynb example 

In [ ]:
methane = streamm.Buildingblock('methane')

In [ ]:
methane.read_xyz()

Create the neighbor list 

In [ ]:
methane.bonded_nblist = methane.guess_nblist(0,radii_buffer=1.25)

and the bonded interactions

In [ ]:
methane.bonded_bonds()
methane.bonded_angles()
methane.bonded_dih()

In [ ]:
print methane.n_particles

In [ ]:
print methane.print_properties()

Set the paramkeys so we can identify force field parameters later on

In [ ]:
for pkey,p in methane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

In [ ]:
for pk,p in methane.particles.iteritems():
    p.residue = 1
    p.resname = 'METH'

Set some rsites to be able to join molecules together 

In [ ]:
methane.particles[1].rsite = 'RH'
methane.particles[2].rsite = 'RH'

In [ ]:
methane.find_rsites()

In [ ]:
print methane.show_rsites()

Read in ethane.xyz from the buildinblock.ipynb example 

In [ ]:
ethane = streamm.Buildingblock('ethane')

In [ ]:
ethane.read_xyz()

Guess bonded neighbor list based on `bonded_radii`

In [ ]:
ethane.bonded_nblist = ethane.guess_nblist(0,radii_buffer=1.25)

In [ ]:
ethane.bonded_bonds()
ethane.bonded_angles()
ethane.bonded_dih()

In [ ]:
print ethane.print_properties()

Set the `paramkey`'s  as described in the force field example

In [ ]:
for pkey,p in ethane.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

Set the `resname` of each particle to `ETH` 

In [ ]:
for pk,p in ethane.particles.iteritems():
    p.residue = 1
    p.resname = 'ETH'

Set `rsite`'s to hydrogens to be replaced during join

In [ ]:
ethane.particles[1].rsite = 'RH'
ethane.particles[5].rsite = 'RH'

Run `find_rsites()` to populate `func` list

In [ ]:
ethane.find_rsites()

In [ ]:
print ethane.show_rsites()

In [ ]:
import copy

Create octane from ethane 

Copy ethane to a new Buildingblock octane 

In [ ]:
octane = copy.deepcopy(ethane)

In [ ]:
from streamm.structures.buildingblock import attach

Then attach 3 more ethanes to make an octane 

In [ ]:
for i in range(22):
    octane = attach(octane,ethane,'RH',1,'RH',0)

In [ ]:
octane.tag = 'octane'

In [ ]:
print octane.n_particles

In [ ]:
octane.write_xyz()

Update the tag

In [ ]:
octane.tag = 'octane'

Rename the residue and resname for octane

In [ ]:
for pk,p in octane.particles.iteritems():
    p.residue = 2
    p.resname = "OCT"
 

In [ ]:
octane.write_xyz()

Print new `rsite`'s 

In [ ]:
print octane.show_rsites()

Find the 4th carbon to attach an ethane 

In [ ]:
print octane.particles[14].symbol

In [ ]:
octane.particles[14].rsite = 'R2'

In [ ]:
octane.find_rsites()

Attach the ethane to the fourth carbon to make 4-ethyloctane

In [ ]:
ethyl_octane = attach(octane,ethane,'R2',0,'RH',0)

In [ ]:
ethyl_octane.tag = '4-ethyloctane'

In [ ]:
ethyl_octane.write_xyz()

Create NWChem Calculation object 

In [ ]:
nwchem_ethyl_octane = streamm.NWChem('nw_ethyl_octane_OPT')

Add calculation to project

In [ ]:
nwchem_example.add_calc(nwchem_ethyl_octane)

Set the structure of the calculation to 4-ethyloctane

In [ ]:
nwchem_ethyl_octane.strucC = ethyl_octane

Set the resource to be local

In [ ]:
nwchem_ethyl_octane.set_resource(res_local)

Make the local directories 

In [ ]:
nwchem_ethyl_octane.make_dir()

Change to the `scratch` directory

In [ ]:
os.chdir(nwchem_ethyl_octane.dir['scratch'])

Copy the template files to the scratch direcotry 

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_ethyl_octane.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'scratch'
nwchem_ethyl_octane.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Read in the template files and add them to the `str` dictionary

In [ ]:
nwchem_ethyl_octane.load_str('templates','nw')        
nwchem_ethyl_octane.load_str('templates','run')

Set the properties dictionary to desired calculation details

In [ ]:
nwchem_ethyl_octane.properties['basis'] = '6-31g'
nwchem_ethyl_octane.properties['method'] = 'UHF'
nwchem_ethyl_octane.properties['charge'] = 0
nwchem_ethyl_octane.properties['spin_mult'] = 1
nwchem_ethyl_octane.properties['task'] = 'SCF optimize'
nwchem_ethyl_octane.properties['coord'] = nwchem_ethyl_octane.strucC.write_coord()

In [ ]:
pprint(nwchem_ethyl_octane.properties)

Replace the keys in the template strings and write the input files 

In [ ]:
nwchem_ethyl_octane.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_ethyl_octane.tag))

Add the input file to the properties to be written into the run file

In [ ]:
nwchem_ethyl_octane.properties['input_nw'] = nwchem_ethyl_octane.files['input']['nw']
nwchem_ethyl_octane.replacewrite_prop('run','scripts','run','%s.sh'%(nwchem_ethyl_octane.tag))

Add the log file to the files dictionary 

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_ethyl_octane.tag)
nwchem_ethyl_octane.add_file(file_type,file_key,file_name)

Change back to the root directory and write a json file

In [ ]:
os.chdir(nwchem_example.dir['home'])
nwchem_example.dump_json()

Change back to scratch

In [ ]:
print nwchem_example.dir['scratch']

In [ ]:
os.chdir(nwchem_example.dir['scratch'])

Run the bash script for the calculation or submit the job to the cluster

In [ ]:
print nwchem_ethyl_octane.tag

In [ ]:
nwchem_ethyl_octane.run()

Check the status of all the calculations in the project

In [ ]:
nwchem_example.check()

Run the analysis

In [ ]:
nwchem_example.analysis()

Tar and zip the results and copy them to a storage location

In [ ]:
nwchem_example.store()

Save json in home directory

In [ ]:
os.chdir(nwchem_example.dir['home'])
nwchem_example.dump_json()